In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [21]:
# base article url
base_article = 'http://www.sermonindex.net/modules/articles/'

In [2]:
def create_soup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    soup = BeautifulSoup(requests.get(url, headers=headers).content, 'html.parser')
    return soup

url = 'http://www.sermonindex.net/modules/articles/'

soup = create_soup(url)

In [5]:
soup.find('table', {'bgcolor': '#B6C7BE'}).findNext('table').findAll('td')

[<td width="15"> </td>, <td valign="top">
 <table><td>
 <a href="index.php?view=category&amp;cid=506"><img src="http://img.sermonindex.net/sermonindex1/cat.gif"/></a>
 </td>
 <td valign="top">
 <b><a href="index.php?view=category&amp;cid=506">
 		A Collection of Hymns</a></b></td></table></td>, <td>
 <a href="index.php?view=category&amp;cid=506"><img src="http://img.sermonindex.net/sermonindex1/cat.gif"/></a>
 </td>, <td valign="top">
 <b><a href="index.php?view=category&amp;cid=506">
 		A Collection of Hymns</a></b></td>]

In [19]:
import re
table = soup.find('table', {'bgcolor': '#B6C7BE'})
serm = table.findAllNext('a', href=re.compile('index.php'))

sermon_next_links = [a['href'] for a in serm]

In [22]:
sermon_next_links[0]

'index.php?view=category&cid=506'

In [30]:
import urllib.parse as urlparse

In [34]:
article_url = urlparse.urljoin(base_article, sermon_next_links[0])

In [35]:
soup = create_soup(article_url)

In [36]:
article_url

'http://www.sermonindex.net/modules/articles/index.php?view=category&cid=506'

In [42]:
def parse_article_link(article_soup):
    article_url = article_soup.find('a')['href']
    article_title = article_soup.find('a').text.strip()
    return (article_url, article_title)


article_table = soup.find('table', {'width': '100%'})
all_articles = [parse_article_link(art) for art in article_table.findAll('strong')]

In [43]:
all_articles[0]

('index.php?view=article&aid=8084', 'COME, thou Conqueror of the nations,')

In [67]:
def parse_next_page(soup):
    next_pages = soup.findAll('center')[-2]
    # get all next page links
    all_next_page_links = [a['href'] for a in next_pages.findAll('a')]
    # get previous next page if available
    previous_page_link = next_pages.find('a', text=re.compile('Previous'))
    if isinstance(previous_page_link, dict):
        previous_page_link = previous_page_link.get('href')
    if previous_page_link:
        # there is the 0 index for the previous page and the actual previous page link
        # we need the actual previous page index and increment it by 1 to get the next page
        cur_page_index = [index for index, l_item in enumerate(all_next_page_links) 
                          if l_item == previous_page_link][-1]
        
        next_page_index = cur_page_index + 1

        if next_page_index >= len(all_next_page_links):
            next_page_url = None
        else:
            next_page_url = all_next_page_links[next_page_index]
    else:
        next_page_url = all_next_page_links[0]
    

SyntaxError: invalid syntax (<ipython-input-67-0f168811cfdb>, line 17)

In [81]:
# soup = create_soup('http://www.sermonindex.net/modules/articles/index.php?view=category&cid=506&page=2')
soup = create_soup('http://www.sermonindex.net/modules/articles/index.php?view=category&cid=506&page=0')

In [73]:
next_pages = soup.findAll('center')[-2]
previous_page_link = next_pages.find('a', text=re.compile('Previous'))
if isinstance(previous_page_link, dict):
    link = previous_page_link.get('href')

In [82]:
next_pages.find('a', text=re.compile('Previous')).get('href')

'index.php?view=category&cid=506&page=1'

In [53]:
all_next_page_links = [a['href'] for a in next_pages.findAll('a')]

In [59]:
list(reversed(all_next_page_links)).index(previous_page_link)

30

In [66]:
# there is the 0 index for the previous page and the actual previous page link
# we need the actual previous page index and increment it by 1 to get the next page
cur_page_index = [index for index, l_item in enumerate(all_next_page_links) 
                  if l_item == previous_page_link][-1]

In [65]:
next_page_index = cur_page_index + 1

if next_page_index >= len(all_next_page_links):
    raise ValueError('Exhausted next pages')

3